In [ ]:
!pip install selenium

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 384.9/384.9 KB 30.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 KB 6.7 MB/s eta 0:00:00


In [ ]:
%%shell

# Add debian buster
cat > /etc/apt/sources.list.d/debian.list <<'EOF'
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster.gpg] http://deb.debian.org/debian buster main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-buster-updates.gpg] http://deb.debian.org/debian buster-updates main
deb [arch=amd64 signed-by=/usr/share/keyrings/debian-security-buster.gpg] http://deb.debian.org/debian-security buster/updates main
EOF

# Add keys
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
apt-key adv --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A

apt-key export 77E11517 | gpg --dearmour -o /usr/share/keyrings/debian-buster.gpg
apt-key export 22F3D138 | gpg --dearmour -o /usr/share/keyrings/debian-buster-updates.gpg
apt-key export E562B32A | gpg --dearmour -o /usr/share/keyrings/debian-security-buster.gpg

# Prefer debian repo for chromium* packages only
# Note the double-blank lines between entries
cat > /etc/apt/preferences.d/chromium.pref << 'EOF'
Package: *
Pin: release a=eoan
Pin-Priority: 500


Package: *
Pin: origin "deb.debian.org"
Pin-Priority: 300


Package: chromium*
Pin: origin "deb.debian.org"
Pin-Priority: 700
EOF


Executing: /tmp/apt-key-gpghome.LOC0naWnXe/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys DCC9EFBF77E11517
gpg: key DCC9EFBF77E11517: public key "Debian Stable Release Key (10/buster) <debian-release@lists.debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.SsGarxmFUY/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 648ACFD622F3D138
gpg: key DC30D7C23CBBABEE: public key "Debian Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1
Executing: /tmp/apt-key-gpghome.ogxOJwj2vQ/gpg.1.sh --keyserver keyserver.ubuntu.com --recv-keys 112695A0E562B32A
gpg: key 4DFAB270CAA96DFA: public key "Debian Security Archive Automatic Signing Key (10/buster) <ftpmaster@debian.org>" imported
gpg: Total number processed: 1
gpg:               imported: 1


In [ ]:
!apt-get update
!pip install webdriver_manager
!pip install pyvirtualdisplay
!sudo apt-get install xvfb
!apt-get install chromium chromium-driver
!pip install undetected_chromedriver

Get:1 http://deb.debian.org/debian buster InRelease [122 kB]
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://deb.debian.org/debian buster-updates InRelease [56.6 kB]
Get:4 http://deb.debian.org/debian-security buster/updates InRelease [34.8 kB]
Hit:5 http://archive.ubuntu.com/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:8 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Get:9 http://deb.debian.org/debian buster/main amd64 Packages [10.7 MB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:12 http://deb.debian.org/debian buster-updates/main amd64 Packages [9,745 B]
Get:13 http://deb.debian.org/debian-security buster/updates/main amd64 Packages [602 kB]
Hit:14 http://ppa.l

In [ ]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service 
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chromium.webdriver import ChromiumDriver
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import csv
import pandas as pd
import undetected_chromedriver
from pyvirtualdisplay import Display


In [ ]:
display = Display(visible=0, size=(1920, 1200))  
display.start()
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('start-maximized')
options.add_argument('enable-automation')
options.add_argument('--disable-infobars')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--disable-browser-side-navigation')
options.add_argument("--remote-debugging-port=9222")
# options.add_argument("--headless")
options.add_argument('--disable-gpu')
options.add_argument("--log-level=3")
driver = webdriver.Chrome( options=options)

In [ ]:
keyword ="data scientist junior"
keyword2 = "data analyst junior"
keyword3 = 'data scientist junior or data analyst junior'
# меняем keyword по необходимости 
url = f"https://www.linkedin.com/jobs/search?_l=en_US&keywords={keyword}&location=%D0%92%20%D0%BB%D1%8E%D0%B1%D0%BE%D0%B9%20%D1%81%D1%82%D1%80%D0%B0%D0%BD%D0%B5&locationId=&geoId=92000000&sortBy=R&f_TPR=r86400&position=1&pageNum=0"

driver.get(url)
sleep(3)

In [ ]:
%%time 
# scroling до кнопки еше
cont = driver.find_element(By.XPATH,f'/html/body/div[1]/div/main/section[2]/button')
for i in range(10):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    sleep(1)
# нажимаем на кнопку еше и скролим с запасом все равно больше 1000 вакансии не загружается
try:
    for i in range(0,50):
        sleep(1)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        cont.click()
except:
    print("OK")

CPU times: user 522 ms, sys: 67 ms, total: 589 ms
Wall time: 1min 15s


In [ ]:
# достаем ссылки записиваем в список
soup=BeautifulSoup(driver.page_source,"lxml")
link=soup.find_all("a",class_="base-card__full-link")
links=[]
for l in link:
    links.append(l.get("href")+"&_l=en_US") # заставляем говарить по английски

In [ ]:
len(links)# смотрим что наделали

978

In [ ]:
# хедер для реквеста (пока фейковые не понадовались )
# функция скана кадой вакансии на вход будем подавать ссылки из links
# join и split помагают привести текст в адекватный вид, удаляем лишние переводы строк и знаки
# все делаем через try и при ошибках задаем значение unknown

headers={'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/53.0.2785.143 Safari/537.36'}


def scan(ur):
    # try:
    response = requests.get(ur,headers=headers)
    sleep(0.5)                  # можно уменшить время но ошибок больше станет в датафрейме
    soup = BeautifulSoup(response.text,"lxml")
    # except:
    #     print("err request")
    try:
        company = " ".join(soup.find("a",class_="topcard__org-name-link topcard__flavor--black-link").text.split())
    except:
        company = "unknown"
    try:
        title = soup.find("h1",class_="top-card-layout__title font-sans text-lg papabear:text-xl font-bold leading-open text-color-text mb-0 topcard__title").text
    except:
        title = "unknown"
#####################################################
    try:
        xxx = soup.find_all("ul",class_="description__job-criteria-list")
        for ii in xxx:
            y = " ".join(ii.text.split())
    except:
        print(" X problema")
    try:
        ind = " ".join(y.split('Industries')[1].split())
        y = y.split('Industries')[0]
    except:
        ind = "unknown"
    try:
        job = " ".join(y.split('Job function')[1].split())
        y = y.split('Job function')[0]
    except:
        job ="unknown"
    try:
        emp = " ".join(y.split('Employment type')[1].split())
        y = y.split('Employment type')[0]
    except:
        emp = "unknown"
    try:
        jun = " ".join(y.split('Seniority level')[1].split())
    except:
        jun="unknown"
#####################################
    try:
        loc = " ".join(soup.find("span",class_="topcard__flavor topcard__flavor--bullet").text.split())
    except:
        loc = "unknown"
    try:
        time = " ".join(soup.find("span",class_="posted-time-ago__text posted-time-ago__text--new topcard__flavor--metadata").text.split())
    except:
        time = "unknown"
    try:
        link = ur
    except:
        print("da nu na")
    try:
        card_text = " ".join(soup.find("div",class_="show-more-less-html__markup show-more-less-html__markup--clamp-after-5").text.split())
    except:
        card_text = "unknown"
    try:
        country = loc.split(',')[-1]
    except:
        country = "unknown"
    return ([title,company,country,jun,job,emp,ind,loc,time,link,card_text])



In [ ]:
# проходим по links.на вход даем ссылку ответ получаем список значений со страници
%%time
data = []
for i in links:
    data.append(scan(i))
    print(data[-1])  # для отладки 

['Junior Data Scientist', 'Evolven Software', 'Slovenia', 'unknown', 'unknown', 'Full-time', 'unknown', 'Slovenia', '4 hours ago', 'https://si.linkedin.com/jobs/view/junior-data-scientist-at-evolven-software-3553545893?refId=89wxoL8SB3u1Yks8fgtX3w%3D%3D&trackingId=v5VUPGTHO5Nb%2BpHNSF5mVQ%3D%3D&position=1&pageNum=0&trk=public_jobs_jserp-result_search-card&_l=en_US', 'We are looking for a Junior Data Scientist with experience in Data Science, strong analytical skills, and a passion to take part in R&D. You will use Machine Learning to develop insightful and predictive solutions for our core products. You will gain experience working with large amounts of highly structured data, applied ML, and Anomaly Detection. You will be developing data-driven solutions to create algorithms while working closely with the engineering team to deliver ML models in production, as well as embedding the results into the product.Responsibilities:Work with Product Managers, Engineers, and Customer Success to

In [ ]:
#data[1][8]
from datetime import timedelta, datetime
for jj in range(len(data)):
    if data[jj][8] != "unknown":
        dell = data[jj][8].split()
        if 'hour' in dell[1]:
            now = datetime.now()
            delta = timedelta(hours=int(dell[0]))
            tim = now - delta
            data[jj][8]=tim.date()
            #print(tim.date())
        elif 'minute' in dell[1]:
            now = datetime.now()
            delta = timedelta(minutes=int(dell[0]))
            tim = now - delta
            data[jj][8]=tim.date()
            #print(tim.date())
        elif 'day' in dell[1]:
            now = datetime.now()
            delta = timedelta(days=int(dell[0]))
            tim = now - delta
            data[jj][8]=tim.date()
            #print(tim.date())

In [ ]:
# преоброзуем все в датафрейм
df=pd.DataFrame(data,columns=['title','company','country','Seniority_level','Job_function','employment_type','Industries','location','time','link','text'])


In [ ]:
df

,title,company,country,Seniority_level,Job_function,employment_type,Industries,location,time,link,text
0,Junior Data Scientist,Evolven Software,Slovenia,unknown,unknown,Full-time,unknown,Slovenia,2023-04-04,https://si.linkedin.com/jobs/view/junior-data-...,We are looking for a Junior Data Scientist wit...
1,Junior Data Scientist,BNP Paribas Fortis,Belgium,Entry level,"Analyst, Finance, and Information Technology",Full-time,"Banking, Insurance, and Financial Services","Brussels, Brussels Region, Belgium",2023-04-04,https://be.linkedin.com/jobs/view/junior-data-...,Junior Data Scientist – M/F/XWant to help shap...
2,Científico/a de Datos Júnior,NTT DATA Europe & Latam,Spain,Entry level,"Consulting, Engineering, and Information Techn...",Full-time,IT Services and IT Consulting and Business Con...,"Barcelona, Catalonia, Spain",2023-04-04,https://es.linkedin.com/jobs/view/cient%C3%ADf...,En NTT Data estamos ampliando nuestro equipo d...
3,Machine Learning Engineer,Rose AI,NY,Entry level,Engineering and Information Technology,Full-time,Software Development,"New York, NY",2023-04-03,https://www.linkedin.com/jobs/view/machine-lea...,About Rose AIRose AI is a cutting-edge cloud d...
4,Data Scientist,Covera Health,NY,Entry level,Engineering and Information Technology,Full-time,Hospitals and Health Care,"New York, NY",2023-04-03,https://www.linkedin.com/jobs/view/data-scient...,"About The CompanyAt Covera, we're committed to..."
...,...,...,...,...,...,...,...,...,...,...,...
973,Senior .Net Developer,Diverse Talent Search,United Kingdom,Mid-Senior level,Information Technology and Engineering,Full-time,"Advertising Services, Design Services, and Tec...",United Kingdom,2023-04-04,https://uk.linkedin.com/jobs/view/senior-net-d...,Role: Senior Web Developer ( C#/.Net)Office lo...
974,Junior Systems Engineer Intern (Summer) - Abil...,AMERICAN SYSTEMS,MD,Internship,Information Technology,Contract,IT Services and IT Consulting,"Carderock, MD",2023-04-03,https://www.linkedin.com/jobs/view/junior-syst...,Job DescriptionAre you an aspiring Systems Eng...
975,Java Developer,FNZ Group,United Kingdom,Entry level,Engineering and Information Technology,Full-time,Financial Services,"Newcastle, England, United Kingdom",2023-04-04,https://uk.linkedin.com/jobs/view/java-develop...,"Unlike other organisations, FNZ encourages emp..."
976,Senior Software Engineer,LSEG (London Stock Exchange Group),Thailand,Mid-Senior level,Engineering and Information Technology,Full-time,IT Services and IT Consulting and Financial Se...,"Bangkok, Bangkok City, Thailand",2023-04-04,https://th.linkedin.com/jobs/view/senior-softw...,"In this role, you will develop core frameworks..."


In [ ]:
driver.quit()#закрываем селениум

In [ ]:
#при необходимости сохроняем датафпейм в drive 

# from google.colab import drive
# drive.mount("/content/drive")
# # !mkdir /content/drive/MyDrive
# df.to_csv (r'/content/drive/MyDrive/data2.csv', index = None, header=True, sep='\t', encoding='utf-8')